In [1]:
import pandas as pd

In [2]:
file_path = 'data/PU_BaseTable.xlsx'
df = pd.read_excel(file_path)

selected_columns = [
    'Purchasing Document Number [EKKO_EBELN]', 'Item Number of Purchasing Document [EKPO_EBELP]', 'Company Code [EKKO_BUKRS]', 'Name of Company Code or Company [T001_BUTXT]', 'Currency Key [T001_WAERS]',
    'Document Category Description [DD07T_BSTYP_DDTEXT]', 'Purchasing Document Type [EKKO_BSART]', 'Short Description of Purchasing Document Type [T161T_BATXT]', 'Creation Date of Purchasing Document [EKKO_AEDAT]', 'Purchase Order Creator [c_EKKO_ERNAM]',
    'Purchase Order Creator Department [c_EKKO_ERNAM_DEPARTMENT]', 'Vendor\'s account number [EKKO_LIFNR]', 'Name 1 [LFA1_NAME1]', 'Vendor Creation Date [c_LFA1_ERDAT]', 'Description of purchasing group [T024_EKNAM]',
    'PO Approver [c_PO_Approver]', 'Short Text [EKPO_TXZ01]', 'Material Group Description [T023T_WGBEZ]', 'Net PO Value in Reporting Currency [c_EKPO_NETWR_RC]', 'Text for Item Category [T163Y_PTEXT]',
    'Invoice Receipt Value in Reporting Currency [c_EKBE_DMBTR_2_RC]', 'Invoice Receipt First Document Date [c_EKBE_BLDAT_First_2]', 'Invoice Receipt First Posting Date [c_EKBE_BUDAT_First_2]'
]

df_selected = df[selected_columns]

# df_selected
# df_selected[df_selected['Purchasing Document Number [EKKO_EBELN]'] == 4504581776]
# df_selected[df_selected['Purchasing Document Number [EKKO_EBELN]'] == 4504581777]

Pivot Table approach

In [3]:
# Create a pivot table to group by 'Purchasing Document Number [EKKO_EBELN]' and sum relevant columns
pivot_df = df_selected.pivot_table(
    index='Purchasing Document Number [EKKO_EBELN]',
    values=[
        'Net PO Value in Reporting Currency [c_EKPO_NETWR_RC]',
        'Invoice Receipt Value in Reporting Currency [c_EKBE_DMBTR_2_RC]'
    ],
    aggfunc='sum'
).reset_index()

# Merge the pivot table with the first occurrence of the remaining columns
first_occurrence_df = df_selected.drop_duplicates(subset='Purchasing Document Number [EKKO_EBELN]')

result_df = pd.merge(first_occurrence_df, pivot_df, on='Purchasing Document Number [EKKO_EBELN]', suffixes=('', '_summed'))

# Save the resulting dataframe to a new Excel file
# result_df.to_excel('data/PU_Pivot_GroupedTable.xlsx', index=False)

In [5]:
# Add new columns
result_df['Invoice Received'] = result_df['Invoice Receipt First Document Date [c_EKBE_BLDAT_First_2]'].notna()
result_df.head()

,Purchasing Document Number [EKKO_EBELN],Item Number of Purchasing Document [EKPO_EBELP],Company Code [EKKO_BUKRS],Name of Company Code or Company [T001_BUTXT],Currency Key [T001_WAERS],Document Category Description [DD07T_BSTYP_DDTEXT],Purchasing Document Type [EKKO_BSART],Short Description of Purchasing Document Type [T161T_BATXT],Creation Date of Purchasing Document [EKKO_AEDAT],Purchase Order Creator [c_EKKO_ERNAM],...,Short Text [EKPO_TXZ01],Material Group Description [T023T_WGBEZ],Net PO Value in Reporting Currency [c_EKPO_NETWR_RC],Text for Item Category [T163Y_PTEXT],Invoice Receipt Value in Reporting Currency [c_EKBE_DMBTR_2_RC],Invoice Receipt First Document Date [c_EKBE_BLDAT_First_2],Invoice Receipt First Posting Date [c_EKBE_BUDAT_First_2],Invoice Receipt Value in Reporting Currency [c_EKBE_DMBTR_2_RC]_summed,Net PO Value in Reporting Currency [c_EKPO_NETWR_RC]_summed,Invoice Received
0,4504533472,10,5211,Hiestand Germany,EUR,Purchase Order,ZUD,ARYZTA StockTransDel,2024-05-21,RHOFMANN,...,VP SI 85G CHNKY CHOC SUPRM,Cookie,0.00,Stock transfer,NaN,NaN,NaN,0.00,0.00,False
1,4504581776,10,5211,Hiestand Germany,EUR,Purchase Order,NB,Standard PO,2024-04-10,EGNEBNER,...,"Baldor, Part.Nr. VEM3546 f. Oszillation",Spare Parts,484.00,Standard,484.00,2024-04-24,2024-05-14,554.18,554.18,True
2,4504581777,30,5211,Hiestand Germany,EUR,Purchase Order,NB,Standard PO,2024-04-29,GGEILING,...,TK Stiefel Cofra 43,Human Resources,88.99,Standard,88.99,2024-04-30,2024-05-03,219.10,220.06,True
3,4505858980,10,5211,Hiestand Germany,EUR,Purchase Order,NB,Standard PO,2024-05-15,CMICHL,...,SG1 White Confectionery Chunks,Cocoa/Choc-Chocolate,56100.00,Standard,56100.00,2024-06-18,2024-06-19,56100.00,56100.00,True
4,4505986415,10,5211,Hiestand Germany,EUR,Purchase Order,NB,Standard PO,2024-04-02,CMICHL,...,Choc Chunks .5 x.5 x.125 semi sweet,Cocoa/Choc-Chocolate,16358.17,Standard,16358.17,2024-04-16,2024-04-17,16358.17,16358.17,True


In [ ]:
def determine_po_after_invoice(row):
    if not row['Invoice Received']:
        return 'N/A'
    elif pd.to_datetime(row['Creation Date of Purchasing Document [EKKO_AEDAT]']) > pd.to_datetime(row['Invoice Receipt First Document Date [c_EKBE_BLDAT_First_2]']):
        return True
    else:
        return False

result_df['PO After Invoice'] = result_df.apply(determine_po_after_invoice, axis=1)

result_df.head(10)

,Purchasing Document Number [EKKO_EBELN],Item Number of Purchasing Document [EKPO_EBELP],Company Code [EKKO_BUKRS],Name of Company Code or Company [T001_BUTXT],Currency Key [T001_WAERS],Document Category Description [DD07T_BSTYP_DDTEXT],Purchasing Document Type [EKKO_BSART],Short Description of Purchasing Document Type [T161T_BATXT],Creation Date of Purchasing Document [EKKO_AEDAT],Purchase Order Creator [c_EKKO_ERNAM],...,Material Group Description [T023T_WGBEZ],Net PO Value in Reporting Currency [c_EKPO_NETWR_RC],Text for Item Category [T163Y_PTEXT],Invoice Receipt Value in Reporting Currency [c_EKBE_DMBTR_2_RC],Invoice Receipt First Document Date [c_EKBE_BLDAT_First_2],Invoice Receipt First Posting Date [c_EKBE_BUDAT_First_2],Invoice Receipt Value in Reporting Currency [c_EKBE_DMBTR_2_RC]_summed,Net PO Value in Reporting Currency [c_EKPO_NETWR_RC]_summed,Invoice Received,PO After Invoice
0,4504533472,10,5211,Hiestand Germany,EUR,Purchase Order,ZUD,ARYZTA StockTransDel,2024-05-21,RHOFMANN,...,Cookie,0.00,Stock transfer,NaN,NaN,NaN,0.00,0.00,False,N/A
1,4504581776,10,5211,Hiestand Germany,EUR,Purchase Order,NB,Standard PO,2024-04-10,EGNEBNER,...,Spare Parts,484.00,Standard,484.00,2024-04-24,2024-05-14,554.18,554.18,True,False
2,4504581777,30,5211,Hiestand Germany,EUR,Purchase Order,NB,Standard PO,2024-04-29,GGEILING,...,Human Resources,88.99,Standard,88.99,2024-04-30,2024-05-03,219.10,220.06,True,False
3,4505858980,10,5211,Hiestand Germany,EUR,Purchase Order,NB,Standard PO,2024-05-15,CMICHL,...,Cocoa/Choc-Chocolate,56100.00,Standard,56100.00,2024-06-18,2024-06-19,56100.00,56100.00,True,False
4,4505986415,10,5211,Hiestand Germany,EUR,Purchase Order,NB,Standard PO,2024-04-02,CMICHL,...,Cocoa/Choc-Chocolate,16358.17,Standard,16358.17,2024-04-16,2024-04-17,16358.17,16358.17,True,False


In [8]:
result_df['Vendor Created'] = result_df['Vendor Creation Date [c_LFA1_ERDAT]'].notna()
result_df.head(10)


,Purchasing Document Number [EKKO_EBELN],Item Number of Purchasing Document [EKPO_EBELP],Company Code [EKKO_BUKRS],Name of Company Code or Company [T001_BUTXT],Currency Key [T001_WAERS],Document Category Description [DD07T_BSTYP_DDTEXT],Purchasing Document Type [EKKO_BSART],Short Description of Purchasing Document Type [T161T_BATXT],Creation Date of Purchasing Document [EKKO_AEDAT],Purchase Order Creator [c_EKKO_ERNAM],...,Net PO Value in Reporting Currency [c_EKPO_NETWR_RC],Text for Item Category [T163Y_PTEXT],Invoice Receipt Value in Reporting Currency [c_EKBE_DMBTR_2_RC],Invoice Receipt First Document Date [c_EKBE_BLDAT_First_2],Invoice Receipt First Posting Date [c_EKBE_BUDAT_First_2],Invoice Receipt Value in Reporting Currency [c_EKBE_DMBTR_2_RC]_summed,Net PO Value in Reporting Currency [c_EKPO_NETWR_RC]_summed,Invoice Received,PO After Invoice,Vendor Created
0,4504533472,10,5211,Hiestand Germany,EUR,Purchase Order,ZUD,ARYZTA StockTransDel,2024-05-21,RHOFMANN,...,0.00,Stock transfer,NaN,NaN,NaN,0.00,0.00,False,N/A,False
1,4504581776,10,5211,Hiestand Germany,EUR,Purchase Order,NB,Standard PO,2024-04-10,EGNEBNER,...,484.00,Standard,484.00,2024-04-24,2024-05-14,554.18,554.18,True,False,True
2,4504581777,30,5211,Hiestand Germany,EUR,Purchase Order,NB,Standard PO,2024-04-29,GGEILING,...,88.99,Standard,88.99,2024-04-30,2024-05-03,219.10,220.06,True,False,True
3,4505858980,10,5211,Hiestand Germany,EUR,Purchase Order,NB,Standard PO,2024-05-15,CMICHL,...,56100.00,Standard,56100.00,2024-06-18,2024-06-19,56100.00,56100.00,True,False,True
4,4505986415,10,5211,Hiestand Germany,EUR,Purchase Order,NB,Standard PO,2024-04-02,CMICHL,...,16358.17,Standard,16358.17,2024-04-16,2024-04-17,16358.17,16358.17,True,False,True
5,4506004561,10,5211,Hiestand Germany,EUR,Purchase Order,ZUD,ARYZTA StockTransDel,2024-01-05,RHOFMANN,...,0.00,Stock transfer,NaN,NaN,NaN,0.00,0.00,False,N/A,False
6,4506016458,10,5211,Hiestand Germany,EUR,Purchase Order,NB,Standard PO,2024-02-07,GGEILING,...,34.65,Standard,NaN,NaN,NaN,0.00,50.45,False,N/A,True
7,4506016462,10,5211,Hiestand Germany,EUR,Purchase Order,NB,Standard PO,2024-02-09,GGEILING,...,1209.00,Standard,1209.00,2024-02-10,2024-02-16,1209.00,1209.00,True,False,True
8,4506123657,10,5211,Hiestand Germany,EUR,Purchase Order,NB,Standard PO,2024-01-26,SKRAUSS,...,11325.00,Standard,11230.62,2024-02-21,2024-02-22,11230.62,11419.38,True,False,True
9,4506135651,10,5211,Hiestand Germany,EUR,Purchase Order,NB,Standard PO,2024-01-26,SKRAUSS,...,3292.21,Standard,3292.14,2024-02-29,2024-03-01,3292.14,3292.21,True,False,True


In [9]:
# Add new column 'Vendor after PO (days)'
def calculate_vendor_after_po_days(row):
    if pd.notna(row['Vendor Creation Date [c_LFA1_ERDAT]']) and pd.to_datetime(row['Vendor Creation Date [c_LFA1_ERDAT]']) > pd.to_datetime(row['Creation Date of Purchasing Document [EKKO_AEDAT]']):
        return (pd.to_datetime(row['Vendor Creation Date [c_LFA1_ERDAT]']) - pd.to_datetime(row['Creation Date of Purchasing Document [EKKO_AEDAT]'])).days
    else:
        return 'N/A'

result_df['Vendor after PO (days)'] = result_df.apply(calculate_vendor_after_po_days, axis=1)

result_df.head(10)

,Purchasing Document Number [EKKO_EBELN],Item Number of Purchasing Document [EKPO_EBELP],Company Code [EKKO_BUKRS],Name of Company Code or Company [T001_BUTXT],Currency Key [T001_WAERS],Document Category Description [DD07T_BSTYP_DDTEXT],Purchasing Document Type [EKKO_BSART],Short Description of Purchasing Document Type [T161T_BATXT],Creation Date of Purchasing Document [EKKO_AEDAT],Purchase Order Creator [c_EKKO_ERNAM],...,Text for Item Category [T163Y_PTEXT],Invoice Receipt Value in Reporting Currency [c_EKBE_DMBTR_2_RC],Invoice Receipt First Document Date [c_EKBE_BLDAT_First_2],Invoice Receipt First Posting Date [c_EKBE_BUDAT_First_2],Invoice Receipt Value in Reporting Currency [c_EKBE_DMBTR_2_RC]_summed,Net PO Value in Reporting Currency [c_EKPO_NETWR_RC]_summed,Invoice Received,PO After Invoice,Vendor Created,Vendor after PO (days)
0,4504533472,10,5211,Hiestand Germany,EUR,Purchase Order,ZUD,ARYZTA StockTransDel,2024-05-21,RHOFMANN,...,Stock transfer,NaN,NaN,NaN,0.00,0.00,False,N/A,False,N/A
1,4504581776,10,5211,Hiestand Germany,EUR,Purchase Order,NB,Standard PO,2024-04-10,EGNEBNER,...,Standard,484.00,2024-04-24,2024-05-14,554.18,554.18,True,False,True,N/A
2,4504581777,30,5211,Hiestand Germany,EUR,Purchase Order,NB,Standard PO,2024-04-29,GGEILING,...,Standard,88.99,2024-04-30,2024-05-03,219.10,220.06,True,False,True,N/A
3,4505858980,10,5211,Hiestand Germany,EUR,Purchase Order,NB,Standard PO,2024-05-15,CMICHL,...,Standard,56100.00,2024-06-18,2024-06-19,56100.00,56100.00,True,False,True,N/A
4,4505986415,10,5211,Hiestand Germany,EUR,Purchase Order,NB,Standard PO,2024-04-02,CMICHL,...,Standard,16358.17,2024-04-16,2024-04-17,16358.17,16358.17,True,False,True,N/A
5,4506004561,10,5211,Hiestand Germany,EUR,Purchase Order,ZUD,ARYZTA StockTransDel,2024-01-05,RHOFMANN,...,Stock transfer,NaN,NaN,NaN,0.00,0.00,False,N/A,False,N/A
6,4506016458,10,5211,Hiestand Germany,EUR,Purchase Order,NB,Standard PO,2024-02-07,GGEILING,...,Standard,NaN,NaN,NaN,0.00,50.45,False,N/A,True,N/A
7,4506016462,10,5211,Hiestand Germany,EUR,Purchase Order,NB,Standard PO,2024-02-09,GGEILING,...,Standard,1209.00,2024-02-10,2024-02-16,1209.00,1209.00,True,False,True,N/A
8,4506123657,10,5211,Hiestand Germany,EUR,Purchase Order,NB,Standard PO,2024-01-26,SKRAUSS,...,Standard,11230.62,2024-02-21,2024-02-22,11230.62,11419.38,True,False,True,N/A
9,4506135651,10,5211,Hiestand Germany,EUR,Purchase Order,NB,Standard PO,2024-01-26,SKRAUSS,...,Standard,3292.14,2024-02-29,2024-03-01,3292.14,3292.21,True,False,True,N/A


In [10]:
# Add new column 'PO after Invoice (days)'
def calculate_po_after_invoice_days(row):
    if pd.notna(row['Invoice Receipt First Document Date [c_EKBE_BLDAT_First_2]']) and pd.to_datetime(row['Creation Date of Purchasing Document [EKKO_AEDAT]']) > pd.to_datetime(row['Invoice Receipt First Document Date [c_EKBE_BLDAT_First_2]']):
        return (pd.to_datetime(row['Creation Date of Purchasing Document [EKKO_AEDAT]']) - pd.to_datetime(row['Invoice Receipt First Document Date [c_EKBE_BLDAT_First_2]'])).days
    else:
        return 'N/A'

result_df['PO after Invoice (days)'] = result_df.apply(calculate_po_after_invoice_days, axis=1)

# Save the resulting dataframe to a new Excel file
result_df.to_excel('data/PU_GroupedTable.xlsx', index=False)

result_df.head(10)

,Purchasing Document Number [EKKO_EBELN],Item Number of Purchasing Document [EKPO_EBELP],Company Code [EKKO_BUKRS],Name of Company Code or Company [T001_BUTXT],Currency Key [T001_WAERS],Document Category Description [DD07T_BSTYP_DDTEXT],Purchasing Document Type [EKKO_BSART],Short Description of Purchasing Document Type [T161T_BATXT],Creation Date of Purchasing Document [EKKO_AEDAT],Purchase Order Creator [c_EKKO_ERNAM],...,Invoice Receipt Value in Reporting Currency [c_EKBE_DMBTR_2_RC],Invoice Receipt First Document Date [c_EKBE_BLDAT_First_2],Invoice Receipt First Posting Date [c_EKBE_BUDAT_First_2],Invoice Receipt Value in Reporting Currency [c_EKBE_DMBTR_2_RC]_summed,Net PO Value in Reporting Currency [c_EKPO_NETWR_RC]_summed,Invoice Received,PO After Invoice,Vendor Created,Vendor after PO (days),PO after Invoice (days)
0,4504533472,10,5211,Hiestand Germany,EUR,Purchase Order,ZUD,ARYZTA StockTransDel,2024-05-21,RHOFMANN,...,NaN,NaN,NaN,0.00,0.00,False,N/A,False,N/A,N/A
1,4504581776,10,5211,Hiestand Germany,EUR,Purchase Order,NB,Standard PO,2024-04-10,EGNEBNER,...,484.00,2024-04-24,2024-05-14,554.18,554.18,True,False,True,N/A,N/A
2,4504581777,30,5211,Hiestand Germany,EUR,Purchase Order,NB,Standard PO,2024-04-29,GGEILING,...,88.99,2024-04-30,2024-05-03,219.10,220.06,True,False,True,N/A,N/A
3,4505858980,10,5211,Hiestand Germany,EUR,Purchase Order,NB,Standard PO,2024-05-15,CMICHL,...,56100.00,2024-06-18,2024-06-19,56100.00,56100.00,True,False,True,N/A,N/A
4,4505986415,10,5211,Hiestand Germany,EUR,Purchase Order,NB,Standard PO,2024-04-02,CMICHL,...,16358.17,2024-04-16,2024-04-17,16358.17,16358.17,True,False,True,N/A,N/A
5,4506004561,10,5211,Hiestand Germany,EUR,Purchase Order,ZUD,ARYZTA StockTransDel,2024-01-05,RHOFMANN,...,NaN,NaN,NaN,0.00,0.00,False,N/A,False,N/A,N/A
6,4506016458,10,5211,Hiestand Germany,EUR,Purchase Order,NB,Standard PO,2024-02-07,GGEILING,...,NaN,NaN,NaN,0.00,50.45,False,N/A,True,N/A,N/A
7,4506016462,10,5211,Hiestand Germany,EUR,Purchase Order,NB,Standard PO,2024-02-09,GGEILING,...,1209.00,2024-02-10,2024-02-16,1209.00,1209.00,True,False,True,N/A,N/A
8,4506123657,10,5211,Hiestand Germany,EUR,Purchase Order,NB,Standard PO,2024-01-26,SKRAUSS,...,11230.62,2024-02-21,2024-02-22,11230.62,11419.38,True,False,True,N/A,N/A
9,4506135651,10,5211,Hiestand Germany,EUR,Purchase Order,NB,Standard PO,2024-01-26,SKRAUSS,...,3292.14,2024-02-29,2024-03-01,3292.14,3292.21,True,False,True,N/A,N/A
